# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning


# Step 1: Feature Extraction Script

In [6]:
import os
import cv2
import pydicom
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from pathlib import Path

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "image_size": 256,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "root_dir": "../input/osic-pulmonary-fibrosis-progression/train",
    # REPLACE THIS PATH with the actual path of your uploaded model in Kaggle
    # /kaggle/input/u-net-classification/pytorch/default/1/epoch_16_f1_0.9021.pth
    "model_weights": "/kaggle/input/u-net-classification/pytorch/default/1/epoch_16_f1_0.9021.pth", 
    "batch_size": 16
}

# ==========================================
# 2. MODEL DEFINITION (Must match training)
# ==========================================
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.double_conv(x)

class StandardUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 1024))
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv_up1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv_up2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv_up3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv_up4 = DoubleConv(128, 64)
        self.outc = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5); x = torch.cat([x, x4], dim=1); x = self.conv_up1(x)
        x = self.up2(x); x = torch.cat([x, x3], dim=1); x = self.conv_up2(x)
        x = self.up3(x); x = torch.cat([x, x2], dim=1); x = self.conv_up3(x)
        x = self.up4(x); x = torch.cat([x, x1], dim=1); x = self.conv_up4(x)
        return torch.sigmoid(self.outc(x))

# ==========================================
# 3. EXTRACTION LOGIC
# ==========================================
def process_patient(patient_id, model):
    path = os.path.join(CONFIG['root_dir'], patient_id)
    files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.endswith('.dcm')])
    
    patient_metrics = {
        'Patient': patient_id,
        'lung_volume_ml': 0.0,
        'fibrosis_mean_hu': -1000.0,
        'fibrosis_std_hu': 0.0,
        'slice_count': len(files)
    }
    
    if not files: return patient_metrics

    # Batch processing for speed
    hu_values_accumulated = []
    
    for i in range(0, len(files), CONFIG['batch_size']):
        batch_files = files[i : i + CONFIG['batch_size']]
        batch_imgs = []
        batch_metadata = []
        
        # Load and Preprocess
        for f in batch_files:
            try:
                dcm = pydicom.dcmread(f)
                img = dcm.pixel_array.astype(np.float32)
                slope = getattr(dcm, 'RescaleSlope', 1)
                intercept = getattr(dcm, 'RescaleIntercept', -1024)
                img = slope * img + intercept
                
                # Metadata for Volume Calc
                slice_thick = float(getattr(dcm, 'SliceThickness', 1.0))
                # Pixel spacing is usually [row, col]
                pixel_spacing = getattr(dcm, 'PixelSpacing', [1.0, 1.0])
                spacing_x, spacing_y = float(pixel_spacing[0]), float(pixel_spacing[1])
                
                batch_metadata.append((slice_thick, spacing_x, spacing_y))
                
                # Resize for U-Net
                img_resized = cv2.resize(img, (CONFIG['image_size'], CONFIG['image_size']))
                
                # Normalize (Same as Training)
                img_norm = np.clip(img_resized, -1000, 400)
                img_norm = (img_norm - (-1000)) / (400 - (-1000))
                batch_imgs.append(img_norm)
                
            except:
                continue
        
        if not batch_imgs: continue
        
        # Prediction
        inp = torch.tensor(np.array(batch_imgs)).unsqueeze(1).float().to(CONFIG['device'])
        with torch.no_grad():
            preds = model(inp)
            preds = (preds > 0.5).float().cpu().numpy().squeeze(1) # (B, 256, 256)
            
        # Calculate Metrics per slice
        for j, mask in enumerate(preds):
            slice_thick, sp_x, sp_y = batch_metadata[j]
            
            # 1. Volume Calculation
            # Scale factor because we resized the image/mask
            # Original Area = Mask_Pixels * (Original_W / 256) * (Original_H / 256) * sp_x * sp_y
            # Simplified approximation: Assume standard scaling, or just use resized pixels as relative volume
            # To be accurate: Volume = count * (sp_x * 256/Org_W) ... it's complex without org dims.
            # Robust approx: Volume = Count * sp_x * sp_y * slice_thick (Assuming 256 IS roughly the field of view)
            # Better: Just sum the raw mask pixels as a "Relative Volume Score" for the regression model
            
            mask_pixels = np.sum(mask)
            # Volume contribution (approximate mL)
            vol = mask_pixels * sp_x * sp_y * slice_thick / 1000.0 
            patient_metrics['lung_volume_ml'] += vol
            
            # 2. Fibrosis Density (HU values inside the mask)
            # We need the original HU values corresponding to the mask
            # Since we resized input, we assume the mask corresponds to the resized HU image
            # Reconstruct Un-normalized HU image for stats
            img_unnorm = batch_imgs[j] * 1400 - 1000 
            
            lung_tissue = img_unnorm[mask == 1]
            if len(lung_tissue) > 0:
                hu_values_accumulated.extend(lung_tissue.tolist())

    # Aggregate HU metrics
    if hu_values_accumulated:
        # Use simple stats (mean/std) of the tissue
        # Taking a random subset if too large to save memory
        if len(hu_values_accumulated) > 10000:
            arr = np.random.choice(hu_values_accumulated, 10000)
        else:
            arr = np.array(hu_values_accumulated)
            
        patient_metrics['fibrosis_mean_hu'] = np.mean(arr)
        patient_metrics['fibrosis_std_hu'] = np.std(arr)
        
    return patient_metrics

# ==========================================
# 4. MAIN EXECUTION
# ==========================================
def extract_features():
    # Load Model
    model = StandardUNet().to(CONFIG['device'])
    try:
        model.load_state_dict(torch.load(CONFIG['model_weights'], map_location=CONFIG['device']))
        print("✅ U-Net loaded successfully.")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    model.eval()
    
    patients = os.listdir(CONFIG['root_dir'])
    results = []
    
    print("🚀 Extracting Biomarkers from all patients...")
    for p in tqdm(patients):
        if os.path.isdir(os.path.join(CONFIG['root_dir'], p)):
            metrics = process_patient(p, model)
            results.append(metrics)
            
    # Save Results
    df = pd.DataFrame(results)
    df.to_csv("biomarkers.csv", index=False)
    print("🎉 Feature Extraction Complete! Saved to 'biomarkers.csv'")
    print(df.head())

if __name__ == "__main__":
    extract_features()

✅ U-Net loaded successfully.
🚀 Extracting Biomarkers from all patients...


  0%|          | 0/176 [00:00<?, ?it/s]

🎉 Feature Extraction Complete! Saved to 'biomarkers.csv'
                     Patient  lung_volume_ml  fibrosis_mean_hu  \
0  ID00015637202177877247924      532.746878       -654.971900   
1  ID00035637202182204917484     1020.789423       -586.062900   
2  ID00290637202279304677843      381.587860       -609.296700   
3  ID00400637202305055099402     1579.392478       -763.694175   
4  ID00073637202198167792918     2033.309111       -681.002325   

   fibrosis_std_hu  slice_count  
0       304.124213          295  
1       281.396531          574  
2       284.104577          240  
3       284.959797          265  
4       304.808541          355  
